In [ ]:
import cv2
import face_recognition
import pandas as pd
from datetime import datetime

# Initialisation des variables
known_face_encodings = []
known_face_names = []
attendance = []

# Charger les images de référence et les noms des personnes
def load_known_faces_and_names():
    # Ajouter ici le chargement des images et des noms
    pass

# Fonction pour marquer la présence
def mark_attendance(name):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    date_str = now.strftime("%Y-%m-%d")
    
    # Définir l'heure de référence
    reference_time = "09:00:00"
    status = "On Time" if current_time <= reference_time else "Late"

    attendance.append({"Name": name, "Time": current_time, "Date": date_str, "Status": status})

# Charger les visages connus et leurs noms
load_known_faces_and_names()

# Capture vidéo
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    rgb_frame = frame[:, :, ::-1]

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        mark_attendance(name)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

# Enregistrer les présences dans un fichier CSV
df = pd.DataFrame(attendance)
df.to_csv('attendance.csv', index=False)
